In [ ]:
import os

import geopandas as gpd
import torch
import torch.nn as nn
import torch.optim as optim
from shapely.geometry import Polygon

from srai.datasets import AirbnbMulticityDataset
from srai.h3 import h3_to_geoseries
from srai.models import Evaluator, Predictor, RegressionBaseModel, Trainer, Vectorizer
from srai.plotting import plot_numeric_data
from srai.regionalizers import H3Regionalizer

In [ ]:
airbnb = AirbnbMulticityDataset()
gdf_airbnb = airbnb.load(os.getenv("HF_TOKEN"))
gdf_airbnb = gdf_airbnb.loc[gdf_airbnb["city"].isin(["paris"])]

In [ ]:
resolution = 8

In [ ]:
train_gdf, test_gdf = airbnb.train_test_split_bucket_regression(gdf_airbnb)
train_gdf, dev_gdf = airbnb.train_test_split_bucket_regression(train_gdf)  # get dev set

In [ ]:
train_gdf, test_gdf = airbnb.train_test_split_spatial_points(gdf_airbnb)
train_gdf, dev_gdf = airbnb.train_test_split_spatial_points(train_gdf)  # get dev set

In [ ]:
vectorizer = Vectorizer(
    gdf_train=train_gdf,
    HF_dataset_object=airbnb,
    embedder_type="Hex2VecEmbedder",
    h3_resolution=resolution,
)

In [ ]:
dataset_airbnb_train = vectorizer.get_dataset(train_gdf)
embedding_size = dataset_airbnb_train["X"].shape[1]

In [ ]:
(train_gdf.shape[0] + test_gdf.shape[0] + dev_gdf.shape[0]) == gdf_airbnb.shape[0]

In [ ]:
dataset_airbnb_test = vectorizer.get_dataset(test_gdf)

In [ ]:
dataset_airbnb_dev = vectorizer.get_dataset(dev_gdf)

In [ ]:
dataset_airbnb_dev

In [ ]:
dataset_airbnb_test

In [ ]:
dataset_airbnb_train

In [ ]:
type(dataset_airbnb_train)

In [ ]:
regression_model = RegressionBaseModel(embedding_size)

In [ ]:
loss_fn = nn.L1Loss()
optimizer = optim.Adam(regression_model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = {
    "batch_size": 32,
    "task": "regression",
    "epochs": 50,
    "device": device,
    "metric2look4": "MAE",
}
trainer = Trainer(
    model=regression_model,
    train_dataset=dataset_airbnb_train,
    eval_dataset=dataset_airbnb_dev,
    optimizer=optimizer,
    loss_fn=loss_fn,
    **args,
)

In [ ]:
model, _, _ = trainer.train()

In [ ]:
evaluator = Evaluator(task="regression", device=device)

In [ ]:
evaluator.evaluate(model, dataset_airbnb_test, return_metrics=False)

In [ ]:
predictor = Predictor("regression", device=device)

In [ ]:
_, hexes, values = predictor.predict(model, dataset_airbnb_test, resolution=resolution)

In [ ]:
original_label = [dataset_airbnb_test[i]["y"] for i in range(len(dataset_airbnb_test))]
original_hexes = [dataset_airbnb_test[i]["X_h3_idx"] for i in range(len(dataset_airbnb_test))]

In [ ]:
polygons = h3_to_geoseries(
    hexes,
)
preds_gdf = gpd.GeoDataFrame(geometry=polygons)
preds_gdf.crs = {"init": "epsg:4326"}
preds_gdf["price"] = [tensor.item() for tensor in values]
preds_gdf["region_id"] = hexes
preds_gdf.index = preds_gdf["region_id"]

original_polygons = h3_to_geoseries(original_hexes)
original_gdf = gpd.GeoDataFrame(geometry=[Polygon(polygon) for polygon in original_polygons])
original_gdf.crs = {"init": "epsg:4326"}
original_gdf["price"] = [tensor.item() for tensor in original_label]
original_gdf["region_id"] = original_hexes
original_gdf.index = original_gdf["region_id"]

In [ ]:
regionalizer = H3Regionalizer(resolution=resolution)
regions = regionalizer.transform(original_gdf)
plot_numeric_data(regions, "price", original_gdf)

In [ ]:
plot_numeric_data(regions, "price", preds_gdf)